In [35]:
from openai import OpenAI
def generate_email_content(user_input):
    

    client = OpenAI(
        base_url="http://127.0.0.1:8080/v1",  # "http://<Your api-server IP>:port"
        api_key="sk-no-key-required"
    )
    completion = client.chat.completions.create(
        model="LLaMA_CPP",
        messages=[
            {"role": "system", "content": "You are Wendy Planter, an AI wedding planner. Your job is to "
                    "email vendors for information and manage previous information. NEVER show that you are an AI, and never add information not stated in the vendor emails; "
                    "do not put [missing info]. Respond naturally and provide complete information wherever possible."},
            {"role": "user", "content": user_input}
        ]
    )
    return completion.choices[0].message.content

def generate_user_content(user_input):
    

    client = OpenAI(
        base_url="http://127.0.0.1:8080/v1",  # "http://<Your api-server IP>:port"
        api_key="sk-no-key-required"
    )
    completion = client.chat.completions.create(
        model="LLaMA_CPP",
        messages=[
            {"role": "system", "content": "You are Wendy Planter, an AI wedding planner. Your job is to "
                    "speak to the bride about information gathered from vendors and manage previous information. NEVER show that you are an AI, and omit missing information;"
                    "do not put [missing info]. Respond naturally and provide complete information wherever possible."},
            {"role": "user", "content": user_input}
        ]
    )
    return completion.choices[0].message.content
# # Example usage
# user_input = "Write an email to this vendor given the following information asking for a quote for their services"
# email_body = generate_email_content(user_input)
# print(email_body)


In [2]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import formatdate

def send_email(sender_email, sender_password, recipient_email, subject, body):
    try:
        # Create message container
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = subject
        msg['Date'] = formatdate(localtime=True)

        # Attach body to the message
        msg.attach(MIMEText(body, 'plain'))

        # Setup the SMTP server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)

        # Send the email
        server.sendmail(sender_email, recipient_email, msg.as_string())
        server.quit()
        print("Email sent successfully!")

        return msg['Date']
    except smtplib.SMTPAuthenticationError:
        print("Failed to authenticate with the SMTP server. Check your email and password.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [4]:
def send_generated_email(sender_email, sender_password, recipient_email, subject, user_input):
    body = generate_email_content(user_input)
    send_email(sender_email, sender_password, recipient_email, subject, body)

# Example usage
send_generated_email('wendyplanterweddings@gmail.com', 'nqel uytt fzxn iqbr', 'nconterno@me.com', 'Quote Request', "Write an email to this vendor given the following information asking for a quote for their services")


Email sent successfully!


In [4]:
import imaplib
import email
from email.header import decode_header
from email.utils import parsedate_to_datetime

def receive_emails(email_user, email_app_password, max_emails=10):
    try:
        # Connect to the mail server
        mail = imaplib.IMAP4_SSL('imap.gmail.com')
        mail.login(email_user, email_app_password)
        mail.select('inbox')

        # Search for all emails in the inbox
        status, data = mail.search(None, 'ALL')
        mail_ids = data[0].split()

        emails = []
        # Fetch the latest emails up to max_emails
        for i in range(min(len(mail_ids), max_emails)):
            latest_email_id = mail_ids[-(i+1)]
            status, data = mail.fetch(latest_email_id, '(RFC822)')

            # Parse the email
            raw_email = data[0][1]
            msg = email.message_from_bytes(raw_email)

            # Decode the email subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding if encoding else "utf-8")

            # Extract the email body
            body = None
            if msg.is_multipart():
                for part in msg.walk():
                    content_type = part.get_content_type()
                    if content_type == "text/plain" and "attachment" not in part.get("Content-Disposition", ""):
                        body = part.get_payload(decode=True).decode()
                        break
            else:
                body = msg.get_payload(decode=True).decode()

            # Extract the email date
            date = parsedate_to_datetime(msg["Date"])

            emails.append({"from": msg["From"], "subject": subject, "body": body, "date": date})

        mail.logout()
        return emails
    except imaplib.IMAP4.error:
        print("Failed to authenticate with the IMAP server. Check your email and app password.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [5]:
def process_incoming_emails(email_user, email_app_password, max_emails=10):
    received_emails = receive_emails(email_user, email_app_password, max_emails)
    print(received_emails)
    # Process each email as needed
    for email in received_emails:
        # Example processing: Print email content
        print(f"From: {email['from']}")
        print(f"Subject: {email['subject']}")
        print(f"Body: {email['body']}")
        # Implement further processing as needed (e.g., extracting quotes, updating database)

# Example usage
process_incoming_emails('wendyplanterweddings@gmail.com', 'nqel uytt fzxn iqbr',)


[{'from': 'Abney Johnson <abneyjohnson0@gmail.com>', 'subject': 'Re: banana6', 'body': "Sure thing\r\n\r\nOn Sat, Jun 22, 2024 at 7:36\u202fPM <wendyplanterweddings@gmail.com> wrote:\r\n\r\n>  Subject: Request for ABC Photography's Wedding Photography Pricing\r\n>\r\n> Dear ABC Photography Team,\r\n>\r\n> I hope this email finds you well. I'm currently in the process of planning\r\n> a wedding and am reaching out to inquire about your photography services.\r\n> I'd be grateful if you could kindly provide me with your pricing\r\n> information for wedding photography packages.\r\n>\r\n> Please let me know if there are any specific details I should provide to\r\n> help you give an accurate quote, such as the number of photographers, hours\r\n> covered, or specific services included.\r\n>\r\n> Looking forward to hearing back from you soon.\r\n>\r\n> Best regards,\r\n> Wendy Planter\r\n> [Your Contact Information]</s>\r\n", 'date': datetime.datetime(2024, 6, 22, 19, 36, 37, tzinfo=datetime.

In [6]:
def wedding_planner_ai_workflow(user_input, sender_email, sender_password, recipient_email, subject):
    # Generate email content using LLM
    email_body = generate_email_content(user_input)
    
    # Send generated email
    send_email(sender_email, sender_password, recipient_email, subject, email_body)
    
    # Process incoming emails (assuming you want to check for responses periodically)
    process_incoming_emails(sender_email, sender_password)

# Example usage
user_input = "Write an email to this vendor given the following information asking for a quote for their services"
wedding_planner_ai_workflow(user_input, 'wendyplanterweddings@gmail.com', 'nqel uytt fzxn iqbr', 'nconterno@me.com', 'Quote Request')


Email sent successfully!
[{'from': 'Abney Johnson <abneyjohnson0@gmail.com>', 'subject': 'Re: banana6', 'body': "Sure thing\r\n\r\nOn Sat, Jun 22, 2024 at 7:36\u202fPM <wendyplanterweddings@gmail.com> wrote:\r\n\r\n>  Subject: Request for ABC Photography's Wedding Photography Pricing\r\n>\r\n> Dear ABC Photography Team,\r\n>\r\n> I hope this email finds you well. I'm currently in the process of planning\r\n> a wedding and am reaching out to inquire about your photography services.\r\n> I'd be grateful if you could kindly provide me with your pricing\r\n> information for wedding photography packages.\r\n>\r\n> Please let me know if there are any specific details I should provide to\r\n> help you give an accurate quote, such as the number of photographers, hours\r\n> covered, or specific services included.\r\n>\r\n> Looking forward to hearing back from you soon.\r\n>\r\n> Best regards,\r\n> Wendy Planter\r\n> [Your Contact Information]</s>\r\n", 'date': datetime.datetime(2024, 6, 22, 19,

In [7]:
def handle_user_request(user_request, sender_email, sender_password, recipient_email):
    subject = "Request for Quote"
    # Generate the initial email content using LLM
    email_body = generate_email_content(user_request)
    # Send the initial email to the vendor
    send_email(sender_email, sender_password, recipient_email, subject, email_body)
    print("Initial email sent.")


In [47]:
def analyze_email_for_information(user_request, email_body):
    # Ensure email_body is not None and is a string
    print('EHUFFEOUFEHOUFHIOFEIOH')
    if email_body and isinstance(email_body, str):
        client = OpenAI(
            base_url="http://127.0.0.1:8080/v1",  # Adjust as needed
            api_key="sk-no-key-required"
        )
        print('email is being analyzed...')
        # Generate a strict prompt for yes/no response
        completion = client.chat.completions.create(
            model="LLaMA_CPP",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are an assistant that helps determine if the requested information is present in the email response. "
                        "You must only respond with 'yes' or 'no'."
                    )
                },
                {
                    "role": "user",
                    "content": (
                        f"User request: {user_request}\n"
                        f"Email body: {email_body}\n"
                        "Does the email body provide the requested information? Respond with 'yes' or 'no' "
                    )
                }
            ],
            max_tokens=1,  # Limit the response to a single token
            temperature=0,  # Set temperature to 0 for deterministic output
        )

        content = completion.choices[0].message.content.strip().lower()
        print("-------------------")
        print(content)

        print("-------------------")
        print(f"User request: {user_request}\n"
                        f"Email body: {email_body}\n"
                        "Does the email body answer the user request? Respond with 'yes' or 'no' "
                  )

        if content == "yes":
            return True, "Email contains relevant information."
        else:
            return False, "Email does not contain relevant information."
    else:
        return False, "Email body is empty or invalid, follow-up needed."


def receive_and_analyze_emails(email_user, email_app_password, max_emails=3):
    # Receive emails
    emails = receive_emails(email_user, email_app_password, max_emails)
    for email in emails:
        print(f"Analyzing email from {email['from']}: {email['subject']}")
        has_info, message = analyze_email_for_information(email['body'])
        if has_info:
            print("Relevant information acquired.")
            return email['body']
        else:
            print(message)
    return None


In [56]:
def send_follow_up_email(sender_email, sender_password, recipient_email):
    follow_up_request = "We didn't receive the quote information we requested. Can you please provide the details?"
    subject = "Follow-up Request for Quote"
    follow_up_body = generate_email_content(follow_up_request)
    send_email(sender_email, sender_password, recipient_email, subject, follow_up_body)
    print("Follow-up email sent.")

def get_original_message(email_content):
    # Split the content by newlines
    lines = email_content.split('\n')
    
    # Find the index of the first forwarded message indicator
    forwarded_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith('On '):
            forwarded_index = i
            break
    
    # If a forwarded message is found, return content up to two lines before it
    if forwarded_index is not None and forwarded_index > 1:
        return '\n'.join(lines[:forwarded_index]).strip()
    else:
        # If no forwarded message is found, return the whole content
        return email_content.strip()
def store_information(info):
    # Placeholder for storing information (e.g., database or file)
    print("Storing information: ", info)

    info = get_original_message(info)
    # write to a file the information
    with open('information.txt', 'a') as f:
        f.write(info+'\n')
    


def generate_follow_up_email_content():
    follow_up_request = "We didn't receive the quote information we requested. Can you please provide the details?"
    return generate_email_content(follow_up_request)

def converse_with_vendor(user_request, sender_email, sender_password, recipient_email):
    count = 0
    original_request = user_request
    user_request = "write an email to a Vendor based upon this requested information from the user: [" +user_request+"] remember they can contact you at wendyplanterweddings@gmail.com"
    # Step 1: Handle initial user request
    subject = "banana7"
    email_body = generate_email_content(user_request)
    last_sent_date = send_email(sender_email, sender_password, recipient_email, subject, email_body)
    print("Initial email sent.")

    # Step 2: Wait and check for email responses (this could be done with a delay or periodic check)
    import time
    time.sleep(10)  # Simulate waiting for a response

    info = None
    while info is None:
        time.sleep(10)
        print("Checking for responses...")
        emails = receive_emails(sender_email, sender_password)
        print(len(emails))
        for email in emails:
            # Check if the email is from the vendor and sent after the last email sent by the LLM
            # print(recipient_email)
            # print(email['from'])
            # get email which will be inside <> and remove <>
            email['from']=email['from'][email['from'].find('<')+1:email['from'].find('>')]
            # print(email['date'])
            # print(parsedate_to_datetime(last_sent_date))
            # print('-------')
            if email['from'] == recipient_email and email['date'] > parsedate_to_datetime(last_sent_date):
                print(f"Analyzing email from {email['from']}: {email['subject']}")
                print('-----')
                print(email['body'])
                print('-----')
                has_info, message = analyze_email_for_information(original_request,email['body'])
                if has_info:
                    print("Relevant information acquired.")
                    info = email['body']
                    # only get lines from start to two lines before the first '>' (using newline as delimiter)
                    info = info[:info.find('>')-1]
                    
                    break
                else:
                    print(message)

                if info is None:
                    if count >3:
                        print('no response')
                        info = 'we did not receive a response from the vendor providing a quote, contact a different vendor.'
                        break
                    count += 1
                    follow_up_body = generate_follow_up_email_content()
                    last_sent_date = send_email(sender_email, sender_password, recipient_email, "Follow-up Request for Quote", follow_up_body)
                    print("Follow-up email sent.")
                    time.sleep(10)  # Simulate waiting for a response

    # Step 3: Store the acquired information
    store_information(original_request+ "-- answer: "+ info)

    # Step 4: Relay information back to the user using the LLM
    response = generate_user_content(f"based upon this reply from the vendor: {info}: give the bride the requested information. ONLY STATE THINGS FROM THE REPLY AND BE EXTREMELY SUCCINCT. DO NOT ADD ANY ADDITIONAL INFORMATION.")
    print(response)

# Example usage
user_request = "Help me find the price of wedding photography from ABC Photography"

converse_with_vendor(user_request, 'wendyplanterweddings@gmail.com', 'nqel uytt fzxn iqbr', 'abneyjohnson0@gmail.com')

Email sent successfully!
Initial email sent.
Checking for responses...
10
Checking for responses...
10
Checking for responses...
10
Checking for responses...
10
Analyzing email from abneyjohnson0@gmail.com: Re: banana7
-----
3000 dollars.

On Mon, Aug 5, 2024 at 11:48 PM <wendyplanterweddings@gmail.com> wrote:

>  Subject: Inquiry about ABC Photography's Wedding Photography Pricing
>
> Dear ABC Photography Team,
>
> I hope this email finds you well! I'm reaching out on behalf of a couple
> who is interested in your photography services for their upcoming wedding.
> They were impressed with your portfolio and would like to know more about
> the pricing for your wedding photography package.
>
> Could you please kindly provide me with the details regarding the cost of
> your wedding photography services? I understand that every package may
> vary, so any information you could share about what is included in each
> package would be greatly appreciated.
>
> Please feel free to contact me at

In [57]:
user_request = "Help me find the price of flowers from FLOWERS R US"

converse_with_vendor(user_request, 'wendyplanterweddings@gmail.com', 'nqel uytt fzxn iqbr', 'abneyjohnson0@gmail.com')

Email sent successfully!
Initial email sent.
Checking for responses...
10
Checking for responses...
10
Checking for responses...
10
Checking for responses...
10
Checking for responses...
10
Checking for responses...
10
Analyzing email from abneyjohnson0@gmail.com: Re: banana7
-----
Hey so our typical packages include deluxe for 2450, premium for 2150, and
our basic package for 2025.

On Mon, Aug 5, 2024 at 11:50 PM <wendyplanterweddings@gmail.com> wrote:

>  Subject: Inquiry about Flower Prices at Flowers R Us
>
> Dear Flowers R Us Team,
>
> I hope this email finds you well. I'm currently working on a wedding
> project and I'd be grateful if you could help me with some information
> regarding your floral services. Specifically, I'm looking for the prices of
> your various flower arrangements.
>
> Could you please provide me with a detailed price list, including the
> names and prices of your most popular arrangements, as well as any custom
> options you offer? I'd also appreciate it if

In [58]:

#function to read information.txt and summarize the information using the llm
def summarize_information():
    with open('information.txt', 'r') as f:
        info = f.read()
    
    client = OpenAI(
        base_url="http://127.0.0.1:8080/v1",  # "http://<Your api-server IP>:port"
        api_key="sk-no-key-required"
    )
    completion = client.chat.completions.create(
        model="LLaMA_CPP",
        messages=[
            {"role": "system", "content": "You are Wendy Planter, an AI wedding planner. Your job is to "
                    "summarize the information you have gathered from vendors. NEVER show that you are an AI, and never add information not stated in the vendor emails; "
                    "do not put [missing info]. Respond naturally and provide complete information wherever possible."},
            {"role": "user", "content": info}
        ]
    )
    return completion.choices[0].message.content

# Example usage
summary = summarize_information()
print(summary)



 For ABC Photography, their pricing for wedding photography is set at $3,000.

As for FLOWERS R US, they offer three different flower packages:

1. The Premium package costs $2,450.
2. The Deluxe package is priced at $2,250.
3. Lastly, their Basic package is available for $2,050.</s>


In [71]:
#get next step for the bride based on the summary
def get_next_step(summary):
    client = OpenAI(
        base_url="http://127.0.0.1:8080/v1",  # "http://<Your api-server IP>:port"
        api_key="sk-no-key-required"
    )
    completion = client.chat.completions.create(
        model="LLaMA_CPP",
        messages=[
            {"role": "system", "content": "You are Wendy Planter, an AI wedding planner. Your job is to "
                    "speak directly to the bride with the next part of the wedding to plan based on the information you have gathered. NEVER show that you are an AI, and never add information not stated in the vendor emails; "
                    "do not put [missing info]. Respond naturally and provide complete information wherever possible."},
            {"role": "user", "content": summary+':[ based on the summary before] respond with the next vendor type for the bride to contact for a quote (e.g. "You should now book the florist/photographer.") (do not say Based on the information provided)'}
        ]
    )
    return completion.choices[0].message.content

# Example usage
next_step = get_next_step(summary)
print(next_step)

 You should now contact a catering company for a quote on food and beverages for your wedding.</s>
